## Práctica de manejo de variables categóricas e imputación de valores perdidos

En esta práctica vamos a aplicar técnicas para tratar con valores perdidos y transformar variables categóricas en variables numéricas para que todas las técnicas implementadas en Scikit-learn puedan utilizarse con toda la información disponible del problema a abordar.  Además, veremos cómo generar Pipelines (cadenas de técnicas aplicadas secuencialmente) y cómo aplicar transformaciones diferentes a las variables en función de sus características. El guión de la práctica es el siguiente:

* [Imputación de valores perdidos](#1)
* [Transformación de variables categóricas a numéricas](#2)
* [Pipelines](#3)
* [Mezcla de transformaciones en función de las características de las variables](#4)

In [134]:
# Importamos algunas de las librerías y clases necesarias para desarrollar la práctica
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from test_helper import Test

En esta práctica vamos a utilizar un dataset que ofrece una buena combinación entre variable continuas y categóricas, [*Automobile Data*](https://archive.ics.uci.edu/ml/datasets/automobile). Es un problema de regresión en el que debemos predecir el precio de un coche en base a diferentes variables de entrada (numéricas y categóricas).

En primer lugar debemos leer los datos, os facilito los nombres de los atributos puesto que en el .csv no están incluidos.

Leer los datos utilizando la función [*read_csv*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) de Pandas (como no hay nombres de variables el parámetro *header* se debe asignar a *None* y se debe indicar que el símbolo *?* en el .csv denota valores perdidos): 

Llamad al DataFrame autos y mostrad los 5 primeros ejemplos.

In [135]:
# Definimos los nombres de los atributos ya que el dataset no los contiene
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

autos = pd.read_csv('automobile.csv',header=None,na_values='?',names=headers)
atSalida = 'price'

In [136]:
Test.assertEquals(list(map(lambda ind: ind, list(autos.symboling.head(5)))), [3, 3, 1, 2, 2], 'Lectura incorrecta')

1 test passed.


En primer lugar, vamos a analizar los tipos de datos almacenados en el DataFrame que acabamos de generar. Para ello se puede visualizar la propiedad *dtypes*.

In [137]:
print(autos.dtypes)

symboling              int64
normalized_losses    float64
make                  object
fuel_type             object
aspiration            object
num_doors             object
body_style            object
drive_wheels          object
engine_location       object
wheel_base           float64
length               float64
width                float64
height               float64
curb_weight            int64
engine_type           object
num_cylinders         object
engine_size            int64
fuel_system           object
bore                 float64
stroke               float64
compression_ratio    float64
horsepower           float64
peak_rpm             float64
city_mpg               int64
highway_mpg            int64
price                float64
dtype: object


Las columnas cuyo tipo de dato sea *object* son posibles variables categóricas. La razón de que solamente sean posibles que si existieran valores perdidos en un atributo numérico y estuviera representado por un string también conllevaría que el tipo de la columna fuera *object*.

# Imputación de valores perdidos <a class="anchor" id="1"></a>

Lo primero que vamos a realizar es ver si la variable a predecir, *price*, contiene valores perdidos o no. En caso de que tenga vamos a eliminar los ejemplos correspondientes puesto que no podemos utilizarlos para resolver nuestro problema de regresión:
* El método [*isnull*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html) de Pandas determina si los valores de una columna son nulos o no. Si sumamos esa máscara obtendremos el número de valores perdidos.

Obtened el número de valores nulos de la variable *price*.

In [138]:
numeroNulosPrice = pd.isnull(autos.price).sum()

In [139]:
Test.assertEquals(numeroNulosPrice, 4, 'Numero de valores nulos de price incorrecto')

1 test passed.


Ahora vamos a obtener los índices de los ejemplos con valores nulos en la variable *price* y los vamos a eliminar.

In [140]:
indicesNull_price = autos[autos[atSalida].isnull()].index.tolist()
print(indicesNull_price)
autos.drop(indicesNull_price, inplace=True)

[9, 44, 45, 129]


Lo primero que vamos a hacer es dividir el DataFrame en dos: uno con las variables de entrada y otro con la de salida.

In [141]:
# DataFrame con la variable de salida
autos_output = autos.price.copy()
# DataFrame con la variable de entrada
autos = autos.drop(['price'], axis=1).copy()

A continuación vamos a imputar los valores perdidos de las diferentes variables de entrada del problema. Para ello, como hemos mencionado anteriormente, podemos utilizar el método *isnull*. En este caso, vamos a visualizar el número de valores nulos de cada variable del DataFrame *autos*. Si aplicamos *isnull* sin especificar ninguna variable y sumamos el resultado (la máscara booleana) obtenemos una *Serie* cuyo índice son los nombres de las variables y los valores son el número de valores nulos de cada una de ellas.

Visualizad el número de valores nulos de cada variable y obtened el número total de valores nulos de este DataFrame (sumad los valores, propiedad *values* de la *Serie* obtenida al visualizar los datos).

In [142]:
# Variables y valores nulos
vars_numNulos = pd.isnull(autos).sum()
# Número total de valores nulos del DataFrame
numNulos = vars_numNulos.sum()

In [143]:
Test.assertEquals(list(map(lambda ind: ind, list(vars_numNulos.values))), [0, 37, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 2, 2, 0, 0], 'Numero de nulos por variable incorrecto')
Test.assertEquals(numNulos, 51, 'Numero de valores nulos incorrecto')

1 test passed.
1 test passed.


Podemos visualizar los ejemplos con valores nulos del siguiente modo.

In [144]:
autos[autos.isnull().any(axis=1)]

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,num_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26
5,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,...,five,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25
7,1,NaN,audi,gas,std,four,wagon,fwd,front,105.8,...,five,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25
14,1,NaN,bmw,gas,std,four,sedan,rwd,front,103.5,...,six,164,mpfi,3.31,3.19,9.0,121.0,4250.0,20,25
15,0,NaN,bmw,gas,std,four,sedan,rwd,front,103.5,...,six,209,mpfi,3.62,3.39,8.0,182.0,5400.0,16,22
16,0,NaN,bmw,gas,std,two,sedan,rwd,front,103.5,...,six,209,mpfi,3.62,3.39,8.0,182.0,5400.0,16,22
17,0,NaN,bmw,gas,std,four,sedan,rwd,front,110.0,...,six,209,mpfi,3.62,3.39,8.0,182.0,5400.0,15,20
27,1,148.0,dodge,gas,turbo,NaN,sedan,fwd,front,93.7,...,four,98,mpfi,3.03,3.39,7.6,102.0,5500.0,24,30


Una vez que sabemos que algunas de las variables de entrada tienen valores perdidos vamos a imputar sus valores mediante la técnica de la imputación de la media (moda, mediana). Para ello Scikit-learn nos ofrece la clase [*SimpleImputer*](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) en la que el híper-parámetro *strategy* define la estrategia de imputación a utilizar:
* *mean*: asigna la media de los ejemplos en cada variable para los valores perdidos
* *median*: asigna la mediana de los ejemplos en cada variable para los valores perdidos
* *most_frequent*: asigna la moda de los ejemplos en cada variable para los valores perdidos

Sin embargo, tenemos un problema y es que existen valores perdidos en variables tanto categóricas como numéricas. Por este motivo, debemos realizar la imputación adecuada a cada tipo de variable.

Para poder aplicar operaciones diferentes a cada grupo de variables, *Scikit-learn* provee la clase [*ColumnTransformer*](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer) mediante la que se pueden aplicar determinadas transformaciones a determinadas variables. Para ello, en el híper-parámetro *transformers* se debe especificar una lista de tuplas en la que en cada tupla se determina el nombre de la transformación, la transformación a aplicar y las variables sobre las que aplicarla:

    (nombreTransformación, transformación, ListaVariables)
    
Una vez creado el objeto de la clase *ColumnTransformer* se pueden aplicar los métodos habituales de las técnicas de pre-procesamiento de datos: *fit*, *transform* y *fit_transform*.

En primer lugar, debemos obtener dos listas con los nombres de las variables que sean categóricas (*dtypes=object*) y numéricas (*dtypes!=object*). De este modo, podremos realizar operaciones diferentes a cada tipo de variable.

# Obtnemos las listas de variables categóricas y numéricas

In [145]:
nombres_variables_numericas = autos.dtypes != object
nombres_variables_categoricas = autos.dtypes == object

A continuación se debe realizar la imputación de valores perdidos de acuerdo al tipo de las variables.

In [146]:
# Importamos las librerías SimpleImputer y ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Las variables numéricas las imputamos con la media y las categóricas con la moda: creamos el objeto de ColumnTransformer
t = [("media", SimpleImputer(strategy='mean'), nombres_variables_numericas), ("moda", SimpleImputer(strategy='most_frequent'), nombres_variables_categoricas)]
preprocesamiento = ColumnTransformer(transformers=t)

# Creamos una copia de autos para poder trabajar posteriormente con los datos originales
autos_sin_missing = autos.copy()
# Entrenamos e imputamos el método de preprocesamiento: imputación apropiada según el tipo de variable
autos_sin_missing = preprocesamiento.fit_transform(autos_sin_missing)

El realizar la transformación anterior, la variable con los datos imputados pasa a ser un array de Numpy. Por este motivo, vamos crear un DataFrame a partir de dicha variable (y tener toda la información de nombres de variables por ejemplo). La único que hay que tener en cuenta es que las variables no estarán en el orden original sino en el orden en el que haya sido utilizadas por *ColumnTransformer*. Por tanto, el parámetro *columns* de *DataFrame* hay que asignarlo de forma adecuada (el *index* será el mismo que el del DataFrame *autos*).

In [147]:
# Creamos un DataFrame a partir de autos_sin_missing
cols = np.concatenate((nombres_variables_numericas[nombres_variables_numericas].index,nombres_variables_categoricas[nombres_variables_categoricas].index))
autos_sin_missing = pd.DataFrame(autos_sin_missing,columns=cols)

In [148]:
Test.assertEquals(list(map(lambda ind: ind, list(autos_sin_missing['normalized_losses'].head()))), [122,122,122,164,164], 'Valores imputados para normalized_losses incorrecto')
Test.assertEquals(list(map(lambda ind: round(ind,2), list(autos_sin_missing['bore'][50:55]))), [3.03,3.08,3.33,3.33,3.33], 'Valores imputados para bore incorrecto')

1 test passed.
1 test passed.


# Transformación de variables categóricas a numéricas <a class="anchor" id="2"></a>

En esta segunda parte de la práctica vamos a abordar un problema habitual en problemas de Data Science, lidiar con las variables categóricas: aquellas que suelen estar almacenadas como texto y que tienen un número finito de valores. El reto es saber cómo utilizar estas variables para poder aplicar técnicas de aprendizaje automático que no las permitan o que la librería utilizada no las permita. Es decir, el reto es saber cómo transformar los valores textuales en valores numéricos.

Afortunadamente, existe una librería en la que están implementadas numerosas técnicas de transformación de variables categóricas a numéricas. Esta librería se llama [*category_encoders*](https://pypi.org/project/category-encoders/) y la debemos instalar: 

    pip install category_encoders
    
En concreto vamos a utilizar los siguientes métodos de transformación de variables categóricas en numéricas:

* Codificación ordinal
* Codificación del conteo
* Codificación One-Hot
* Codificación binaria
* Codificación basada en la salida

Además de todas estas técnicas, también ofrece métodos avanzados como Catboost, el contraste de Helmert, el contraste polinomial, backward difference, etc... 

Las librerías Pandas y Scikit también implementan algunas de estas técnicas pero los procesos son más complicados y, en el caso de las implementaciones de Pandas no permiten incluir esta transformación dentro de una *Pipeline* (la veremos en esta práctica) al no tener implementados los métodos *fit* y *transform*.

In [149]:
# Importamos la librería
import category_encoders as ce

## Codificación ordinal

La codificación ordinal consiste en asignar valores numéricos a las diferentes etiquetas. 

Para realizarlo muy fácilmente, *category_encoders* ofrece la clase [*OrdinalEncoder*](http://contrib.scikit-learn.org/category_encoders/ordinal.html). Los híper-parámetros de esta clase son sencillos, debemos destacar dos:
* *cols*: permite establecer una lista con los nombres de las variables a transformar. Si se usa el valor por defecto, que es *None*, transforma todas las variables categóricas.
* *mapping*: es una lista de diccionarios que permitiría establecer manualmente los valores numéricos a asignar a las diferentes categorías de cada variable. 
    * Esta opción daría lugar a la técnica de transformación de variables categóricas a numericas conocida como *encoding labels*. En Scikit-learn se puede realizar esta transformación utilizando la clase [*LabelEncoder*](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) y en Pandas mediante el método [*replace*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html).
    
NOTA: Pandas permite realizar la transformación ordinal utilizando el método [*factorize*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html).

Cread un DataFrame llamado autos_ord, en el que transforméis todas las variable categóricas utilizando la codificación ordinal. Partid del DataFrame en el que hemos imputado los valores perdidos.

In [150]:
# Creamos una copia del DataFrame para no perder los datos originales y poder seguir trabajando con ellos
autos_ord = autos_sin_missing.copy()

# Realizamos la transformación utilizando la codificación ordinal
myEncoder = ce.ordinal.OrdinalEncoder()
autos_ord = myEncoder.fit_transform(autos_ord)

In [151]:
Test.assertEquals(list(map(lambda ind: ind, list(autos_ord.make.head(5)))), [1, 1, 1, 2, 2], 'Transformacion incorrecta')
Test.assertEquals(list(map(lambda ind: ind, list(autos_ord.engine_type.head(5)))), [1, 1, 2, 3, 3], 'Transformacion incorrecta')

1 test passed.
1 test passed.


## Codificación del conteo

En la codificación del conteo reemplazamos cada valor categórico por el número de apariciones en el conjunto de entrenamiento de dicho valor.

La librería *category_encoders* ofrece la clase [*CountEncoder*](http://contrib.scikit-learn.org/category_encoders/count.html). Los híper-parámetros de esta clase son sencillos y también dispone de *cols* como el método anterior.

Cread un DataFrame, *autos_count*, en el que se transformen todas las variables categóricas de acuerdo a esta codificación. Recordad ralizar una copia al DataFrame en el que se ha realizado la imputación de valores perdidos.

In [152]:
# Realizamos la transformación utilizando la codificación del conteo
autos_count = autos_sin_missing.copy()

# Realizamos la transformación utilizando la codificación
myEncoder = ce.count.CountEncoder()
autos_count = myEncoder.fit_transform(autos_count)

In [153]:
Test.assertEquals(list(map(lambda ind: ind, list(autos_count.make.head(5)))), [3, 3, 3, 6, 6], 'Transformacion incorrecta')

1 test passed.


## Codificación One Hot

En la codificación One Hot, se crean tantas variables nuevas como posibles valores de cada variable categórica. En cada una de estas variables se asignará el valor *True* en caso de que el ejemplo tenga ese valor y *False* en caso contrario.

La librería *category_encoders* ofrece la clase [*OneHotEncoder*](http://contrib.scikit-learn.org/category_encoders/onehot.html) y los híper-parámetros son comunes a las clases anteriores. 
* Pandas permite realizar esta transformación mediante el método [*get_dummies*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html).
* Scikit-learn ofrece la clase [*LabelBinarizer*](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) para realizara esta transformación.

Realizad la transformación de todas las variables categóricas partiendo de una copia del DataFrame en el que hemos realizado la imputación de valores perdidos y almacenad el resultado en un DataFrame llamado *autos_oh*.

In [154]:
# Realizamos la transformación utilizando la codificación One Hot
autos_oh = autos_sin_missing.copy()

# Realizamos la transformación utilizando la codificación
myEncoder = ce.one_hot.OneHotEncoder()
autos_oh = myEncoder.fit_transform(autos_oh)

In [155]:
Test.assertEquals(list(map(lambda ind: ind, list(autos_oh.make_1.head(5)))), [1, 1, 1, 0, 0], 'Transformacion incorrecta')

1 test passed.


## Codificación binaria

La codificación binaria, el igual que la One Hot, aumenta el número de variables de entrada. En este caso, realiza la creación de nuevas variables aplicando primero la codificación ordinal, cada valor lo pasa a binario y luego crea tantas nuevas variables como bits tenga el número binario más grande copiando el número bit a bit.

La librería *category_encoders* ofrece la clase [*BinaryEncoder*](http://contrib.scikit-learn.org/category_encoders/binary.html) y los híper-parámetros son comunes a las clases anteriores. 

Realizad la transformación de todas las variables categóricas partiendo de una copia del DataFrame en el que hemos realizado la imputación de valores perdidos y almacenad el resultado en un DataFrame llamado *autos_binary*.

In [156]:
# Realizamos la transformación utilizando la codificación binaria
autos_binary = autos_sin_missing.copy()

# Realizamos la transformación utilizando la codificación
myEncoder = ce.binary.BinaryEncoder()
autos_binary = myEncoder.fit_transform(autos_binary)

In [157]:
Test.assertEquals(list(map(lambda ind: ind, list(autos_binary.make_4.head(5)))), [0, 0, 0, 1, 1], 'Transformacion incorrecta')

1 test passed.


## Codificación basada en la salida

La codificación basada en la salida es la única de la que hemos visto que tiene en cuenta información de la salida para realizar la transformación.

La librería *category_encoders* ofrece la clase [*TargetEncoder*](http://contrib.scikit-learn.org/category_encoders/targetencoder.html) y los híper-parámetros son comunes a las clases anteriores. Esta clase tiene un híper-parámetro llamado *smoothing* que determina si para asignar el valor se tiene en cuenta los valores de la variable a predecir de todos los ejemplos (independientemente de la categoría a sustituir):
* Regresión: la media de los valores (de la variable a predecir) que sean de la categoría a transformar se modifica teniendo en cuenta la media de los valores de la variable a predecir de todos los ejemplos. 
* Clasificación: la probabilidad de que los ejemplos que sean de la categoría a transformar sean de la clase 1 se modifica teniendo en cuenta la probabilidad de tener dicha clase en el conjunto de entrenamiento.

En ambos caso, cuanto mayor sea el valor del híper-parámetro *smoothing* más tenderá el valor transformado a la media de la variable a predecir de todos los ejemplos (o la probabilidad de tener ejemplos de la clase 1). 

Además, como esta técnica utiliza información de la variable a predecir, al método *fit* se le debe pasar tanto los valores de los ejemplos en las variables de entrada (X) como en la de salida (y) para que pueda aprender los valores a asignar a cada categoría.

Realizad la transformación de todas las variables categóricas partiendo de una copia del DataFrame en el que hemos realizado la imputación de valores perdidos y almacenad el resultado en un DataFrame llamado *autos_te*. Para ello, utilizad 0.0000001 como valor de *smoothing*.

In [158]:
# Realizamos la transformación utilizando la codificación basada en la salida 
autos_te = autos_sin_missing.copy()

# Realizamos la transformación utilizando la codificación
myEncoder = ce.target_encoder.TargetEncoder(smoothing=0.0000001)
autos_te = myEncoder.fit_transform(autos_te,autos_output)

In [159]:
Test.assertEquals(list(map(lambda ind: round(ind,2), list(autos_te.make.head(5)))), [15498.33, 15498.33, 15498.33, 17859.17, 17859.17], 'Transformacion incorrecta')

1 test passed.


## Pipelines <a class="anchor" id="3"></a>

En esta sección vamos a ver una clases que permite establecer una secuencia de técnicas a aplicar a los datos. Es decir, podemos establecer todas las técnicas de pre-procesamiento a aplicar (y en el orden deseado) y finalizar con una técnica de predicción. Una vez que se ha establecido la secuencia, al realizar el aprendizaje, se aplicará el aprendizaje de cada componente de forma secuencial y por tanto, los componentes aprenderán sobre el resultado dado por los componentes previos de la secuencia (lo mismo pasa al realizar la predicción). De este modo, ahorramos código y minimizamos las opciones de cometer fallos de programación y incurrir en *data leakage* que afecten al resultado obtenido. 

La clase que nos ofrece esta posibilidad se llama [*Pipeline*](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) y está dentro de la librería *pipeline* de Scikit-learn. La llamada al constructor de esta clase consiste en un conjunto de tuplas del tipo (nombreFase, objeto), cuyo significado es:
* nombreFase: string que establece el nombre de la fase. Por ejemplo 'tipoCodificacion', 'estandarizacion' o 'clasificador'
* objeto: variable en la que se almacena la llamada al constructor de lo que se desee hacer. Por ejemplo ce.TargetEncoder(smoothing=0.0000001), MinMaxScaler() o neighbors.KNeighborsRegressor()

Es decir, si quisiéramos combinar los procesos mencionados anteriormente deberíamos realizar la siguiente llamada:

    pipeline = Pipeline([('tipoCodificacion', ce.TargetEncoder(smoothing=0.0000001)), ('estandarizacion', MinMaxScaler()),  ('modelo', neighbors.KNeighborsRegressor())])
    
Hay que destacar que los objetos de todas las fases de la secuencia (excepto de la última) tienen que tener los métodos *fit* y *transform*, para que puedan aprender de los datos y transformarlos en consecuencia. El último objeto debe tener el método *fit*, para aprender de los datos, y el método *predict* para poder realizar nuevas predicciones. Es decir, el último objeto debe ser un modelo de clasificación o regresión.

El objeto combinado, la Pipeline generada, dispone de los siguientes métodos:
* fit: Recibe como parámetros de entrada los datos de entrada (X) y de salida (Y). Cada objeto de cada fase aprende en base a dichos datos.
* predict: Recibe como parámetro de entrada los datos de entrada (X). Realiza la predicción realizando lo siguiente:
    * Primero se aplican las transformaciones de datos por medio de los primeros objetos (llaman a sus respectivas funciones transform)
    * Finalmente, se aplica al objeto de la última fase (clasificador o modelo de regresión) para realizar la predicción correspondiente a los datos de entrada (llamada a su método predict). 

Realizar una Pipeline que combine:
* Transformación de variables categóricas a numéricas utilizando la codificación ordinal.
* Estandarización de los datos utilizando el método del mínimo y del máximo.
* Modelo de regresión KNN

Para ello, en primer lugar crea los conjuntos de entrenamiento y test utilizando *train_test_split*, un 20% de los ejemplos como test y con la semilla 12 sobre el DataFrame al que le hemos imputado valores perdidos, *autos_sin_missing*.

Obtened la raíz cuadrada del error cuadrático medio (RMSE) en train y en test.

In [160]:
# Se importa la librería pipeline
from sklearn import pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics, model_selection, neighbors
from sklearn.metrics import mean_squared_error
from math import sqrt


# Creación de los conjuntos de train y de test
X_train, X_test, y_train, y_test = model_selection.train_test_split(
autos_sin_missing, autos_output,train_size=0.8, random_state=12)

# Se crea la Pipeline con las fases deseadas
pipe = pipeline.Pipeline([('tipoCodificacion', ce.ordinal.OrdinalEncoder()), 
                 ('estandarizacion', MinMaxScaler()),  ('modelo', neighbors.KNeighborsRegressor())
                ])
# Se realiza el aprendizaje de los parámetros de todas las fases de la Pipeline
pipe = pipe.fit(X_train,y_train)
# Se llama a la predicción de la pipeline sobre los datos de entrenamiento
prediccionesTrain = pipe.predict(X_train)
# Se calcula la raíz cuadrada del error cuadrático medio en train (con dos decimales)
rmseTrain = round(sqrt(mean_squared_error(y_train.values, prediccionesTrain)),2)
# Se llama a la predicción de la pipeline sobre los datos de test
prediccionesTest = pipe.predict(X_test)
# Se calcula la raíz cuadrada del error cuadrático medio en test (con dos decimales)
rmseTest = round(sqrt(mean_squared_error(y_test.values, prediccionesTest)),2) 
print('Error en entrenamiento {} y en test {}'.format(rmseTrain, rmseTest))

Error en entrenamiento 4531.67 y en test 5102.61


In [161]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
Test.assertEquals(rmseTrain, 4531.67, 'Valor de RMSE en train incorrecto')
Test.assertEquals(rmseTest, 5102.61, 'Valor de RMSE en test incorrecto')

1 test passed.
1 test passed.


Al igual que realizamos para un modelo de aprendizaje, existe la posibilidad de buscar la mejor configuración de toda la Pipeline (los mejores valores de sus híper-parámetros). Para ello, utilizamos la clase llamada *GridSearchCV* de la librería *model_selection* que realiza tal proceso de forma automática.

Una vez importado el paquete podemos usar la clase y para ello lo primero que hay que hacer es una llamada al constructor

    model_selection.GridSearchCV(pipeline, hiperParametros)
    
El resto de parámetros de la clase son los mismos que vimos en prácticas anteriores. Los parámetros indicados en el código anterior son:
* pipeline: el objeto de la clase Pipeline
* hiperParametros: un diccionario con los nombres de los híper-parámetros como claves y los valores de cada uno de ellos como valor. 
    * En este caso, al nombre del campo hay que insertarle como prefijo el componente al que hace referencia seguido de dos barras bajas (nombreComponente__campo: [valores]). 
    * Por ejemplo modelo__n_neighbors: [3,5,7,9]

Una vez generado el objeto, el siguiente paso es realizar el aprendizaje. Para ello se llama al método *fit*. Este paso y la visualización de la mejor configuración los rendimientos asociados es igual a los vistos para la validación de modelos.

Utiliza la clase GridSearchCV para buscar la configuración óptima de la Pipeline creada en la celda anterior. Los híper-parámetros a analizar y sus valores son:
* Para KNN
    * n_neighbos: 3, 5, 7 y 9
    * weights: uniform y distance
    * p: 1, 2, 1.5 y 3
    
Como métrica de rendimiento se debe utilizar *neg_root_mean_squared_error* (es la raíz del error cuadrático medio en negativo, para que un mayor valor sea un mejor modelo). Como metodología de validación de modelos se debe usar Hold-out utilizando un 20% de los ejemplo para validación (para ello se debe usar [*ShuffleSplit*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) de la librería *model_selection* que es la misma que *StratifiedShuffleSplit* pero sin aplicar estatificación puesto que estamos en un problema de regresión). 

Una vez encontrada la mejor configuración de la Pipeline, utilizadla para obtener el rendimiento (RMSE) en entrenamiento y en test.

In [162]:
from sklearn import model_selection

# Se crea la Pipeline con las fases deseadas
# pipe = <RELLENAR>
# Se crea el grid de híper-parámetros a "optimizar"
# hiperParameters = <RELLENAR>
# Se llama al constructor de GridSearchCV para que genere todas las combinaciones ce los parámetros definidos anteriormente
np.random.seed(12)
# gridSearch_pipeline = <RELLENAR>
# Se realiza el aprendizaje de todos los clasificadores considerados (todas las configuraciones)
# gridSearch_ClasCompuesto = <RELLENAR>
# Se muestra la mejor configuración junto con su rendimiento
print(gridSearch_pipeline.best_score_)
print(gridSearch_pipeline.best_params_)

# Almacenamos el DataFrame con los resultados
diccionarioResultados = gridSearch_pipeline.cv_results_

# Se muestra el RMSE obtenido para cada posible combinación de híper-parámetros
# <RELLENAR>

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración (mejor modelo)
# mejorModelo = <RELLENAR>
# rmseTrain = <RELLENAR>
# rmseTest = <RELLENAR>

NameError: name 'gridSearch_pipeline' is not defined

In [ ]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(rmseTrain, 177.01, 'Valor de RMSE en train incorrecto')
Test.assertEquals(rmseTest, 3222.77, 'Valor de RMSE en test incorrecto')

In [ ]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
indice = np.argmax(diccionarioResultados['mean_test_score'])
Test.assertEquals(round(np.max(diccionarioResultados['mean_test_score']), 2), -2323.70, "RMSE de la mejor configuración incorrecto")
Test.assertEquals(diccionarioResultados['params'][indice], {'modelo__n_neighbors': 3, 'modelo__weights': 'distance', 'modelo__p': 1}, "Mejor configuración incorrecta")

## Mezcla de transformaciones en función de las características de las variables <a class="anchor" id="4"></a>

Finalmente, vamos a aplicar todo lo que hemos visto hasta ahora de forma conjunta. Es decir, vamos a partir del conjunto original de ejemplos (con valores perdidos) y vamos a aplicar todas las técnicas necesarias para resolver el problema de predicción. 

En este caso, también vamos a tener en cuenta el número de valores diferentes de las variables categóricas para aplicar un tipo de transformación u otra. El motivo es que puede ser interesante aplicar una transformación ordinal a las variables con dos valores (o menos), una transformación *One hot* a las variables con pocos valores y una basada en la salida para las que tienen muchos y, de esta forma, evitar generar una matriz dispersa.

NOTA: la transformación a aplicar en la clase *ColumnTransformer* puede ser una Pipeline en la que se encadenen varias técnicas a aplicar sobre las variables.

Cread 4 listas con los nombres de las variables que cumplan:
* Sean numéricas (nombres_variables_numericas)
* Sean categóricas cuyo número de valores diferentes sea 2 o menos (nombres_variables_categoricas_2_valores)
* Sean categóricas cuyo número de v
alores diferentes esté entre 3 y 6, incluído (nombres_variables_categoricas_entre3y6_valores)
* Sean categóricas cuyo número de valores diferentes sea mayor que 6 (nombres_variables_categoricas_masDe6_valores)

In [ ]:
# Obtnemos las listas de variables 
# nombres_variables_numericas = <RELLENAR>
# nombres_variables_categoricas_2_valores = <RELLENAR>
# nombres_variables_categoricas_entre3y6_valores = <RELLENAR>
# nombres_variables_categoricas_masDe6_valores = <RELLENAR>

Aplicad a cada tipo de variable las siguientes técnicas:
* Numéricas
    * Imputación de valores perdidos por la media
* Categóricas cuyo número de valores diferentes sea 2 o menos
    * Imputación de valores perdidos por la moda
    * Transforamción ordinal
* Categóricas cuyo número de v alores diferentes esté entre 3 y 6
    * Imputación de valores perdidos por la moda
    * Transformación One Hot
* Categóricas cuyo número de valores diferentes sea mayor que 6
    * Imputación de valores perdidos por la moda
    * Transformación basada en la salida con *smoothing=0.0000001* 
    
A todas las variables, una vez realizadas las transformaciones mencionadas anteriormente, se les aplica la estandarización del mínimo y del máximo y se debe utilizar KNN, con sus valores por defecto, como método de regresión.

NOTA: al realizar una Pipeline, se puede incluir una fase en la que un componente sea un objeto de *ColumnTransformer*.

Aplicad todo el proceso partiendo de los datos originales (DataFrame *autos*) que se deben dividir en el conjunto de entrenamiento y de test (20% de ejemplos para test) utilizando 12 como valor de la semilla. Obtened el rendimiento (RMSE) en entrenamiento y en test.

In [ ]:
# Creación de las Pipelines con las técnicas de pre-procesamiento de las variables categóricas
# categorical_2_imputer_transformer = <RELLENAR>
# categorical_entre3y6_imputer_transformer = <RELLENAR>
# categorical_masDe6_imputer_transformer = <RELLENAR>

# Creación del objeto ColumnTransformer con las transformaciones adecuadas
# preprocesamiento = <RELLENAR>


# Creamos la Pipeline final: pre-procesamiento, estandarizacion y KNN
# pipe = <RELLENAR>


# Particionado de los datos en train y test
# X_train, X_test, y_train, y_test = <RELLENAR>

# Entrenamos y evaluamos el rendimiento de la Pipeline en train y test
# rmseTrain = <RELLENAR>
# rmseTest = <RELLENAR>

In [ ]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
Test.assertEquals(rmseTrain, 2901.02, 'Valor de RMSE en train incorrecto')
Test.assertEquals(rmseTest, 3711.53, 'Valor de RMSE en test incorrecto')

Al igual que en el resto de Pipelines, se pueden conseguir los mejores valores de los híper-parámetros utilizando *GridSearchCV*. Como el objeto final anterior es complicado, para conocer los nombres de los híper-parámetros susceptibles de ser tratados podemos ejecutar la siguiente línea de código.

In [ ]:
# Mostramos los nombres de los híper-parámetros
    # Con ellos se puede generar el diccionario de valores para utilizar GridSearchCV
pipe.get_params().keys()

Buscad los mejores valores de los siguientes híper-parámetros de la Pipeline:
* MinMaxScaler
    * feature_range: (0,1) y (0,5)
* KNN
    * n_neighbors: 3, 5, 7, 9
    * weights: uniform y distance
    * p: 1, 2, 1.5 y 3

Utilizad la misma métrica de rendimiento y la misma validación de modelos que antes.

In [ ]:
# Se crea el grid de híper-parámetros a "optimizar"
# hiperParameters = <RELLENAR>

# Se llama al constructor de GridSearchCV para que genere todas las combinaciones de los parámetros definidos anteriormente
np.random.seed(12)
# gridSearch_pipeline = <RELLENAR>
# Se realiza el aprendizaje de todos los clasificadores considerados (todas las configuraciones)
# gridSearch_pipeline = <RELLENAR>
# Se muestra la mejor configuración junto con su rendimiento
print(gridSearch_pipeline.best_score_)
print(gridSearch_pipeline.best_params_)

# Almacenamos el DataFrame con los resultados
diccionarioResultados = gridSearch_pipeline.cv_results_

# Se muestra el RMSE obtenido para cada posible combinación de híper-parámetros
# <RELLENAR>

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración de la Pipeline
# rmseTrain = <RELLENAR>
# rmseTest = <RELLENAR>

In [ ]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
Test.assertEquals(rmseTrain, 177.01, 'Valor de RMSE en train incorrecto')
Test.assertEquals(rmseTest, 2408.35, 'Valor de RMSE en test incorrecto')

In [ ]:
# ESTA CELDA DARÁ ERROR SI EL RESULTADO NO ES CORRECTO
    # EN CASO CONTRARIO NO TENDRÁ SALIDA
indice = np.argmax(diccionarioResultados['mean_test_score'])
Test.assertEquals(round(np.max(diccionarioResultados['mean_test_score']), 2), -2133.58, "RMSE de la mejor configuración incorrecto")
Test.assertEquals(diccionarioResultados['params'][indice], {'estandarizacion__feature_range': (0, 1), 'modelo__n_neighbors': 5, 'modelo__p': 1, 'modelo__weights': 'distance'}, "Mejor configuración de la Pipeline incorrecta")